In [3]:
import dask
import dask.dataframe as df
import numpy as np
import pandas as pd
from scipy import sparse as sp
from math import log
from collections import Counter
import functools 
import sklearn.metrics as metrics
from itertools import combinations, product
from string import ascii_lowercase

In [55]:
def generate_random_string(length):
    return ''.join(np.random.choice(list(ascii_lowercase), size=length))

def generate_key_chunk(params: dict):
    if params['type'] is 'SocialSecurityNumber':
        intervals = np.linspace(1, 100 - 1, num=params['total_chunks']+1, dtype=int)
        data = np.linspace(intervals[params['chunk_pos']], intervals[params['chunk_pos']+1]-1, num=params['chunk_size'], dtype=int)
        data = np.random.permutation(data)
        data = [str(i).zfill(9) for i in data]
        data = list(map(lambda i: '{}-{}-{}'.format(i[:3], i[3:5], i[5:]), data))
    elif params['type'] is 'String':
        length = np.random.randint(params['min'], params['max'])
        vectorized = np.vectorize(lambda x: '{}{}'.format(generate_random_string(length), x))
        data = vectorized(np.arange(params['chunk_size']*params['chunk_pos'],params['chunk_size']*(1+params['chunk_pos'])))
        data = np.random.permutation(data)
    elif params['type'] is 'Integer':
        data = np.arange(params['min'], params['max'])
        data = np.random.permutation(data)
    elif params['type'] is 'Float' or 'Datetime':
        data = np.arange(params['min'], params['max'], (params['min'] - params['max']) / params['chunk_size'])
    else:
        data = None
    return data

In [56]:
parametros = {}
parametros['type'] = 'String'
parametros['chunk_size'] = 5
parametros['total_chunks'] = 10
parametros['min'] = 2
parametros['max'] = 6
#parametros['chunk_pos'] = 2 
for i in np.arange(parametros['total_chunks']-1):
    parametros['chunk_pos'] = i
    values = generate_key_chunk(parametros)
    print(values)

['nglo3' 'rstm0' 'bqqd4' 'eeem1' 'dxgo2']
['uvo6' 'ssm7' 'kcf5' 'zjd9' 'xkd8']
['vqr10' 'dla13' 'bor14' 'bgf11' 'ywc12']
['srbt17' 'unyg15' 'uqiv19' 'xhyo18' 'fxgi16']
['kafia24' 'hlsrx20' 'xfuwz22' 'ksydm23' 'deqft21']
['ndbs26' 'dtrr25' 'tcxy29' 'dwau27' 'xkpi28']
['imacm30' 'ejeou31' 'enwzh33' 'gnrmj34' 'mufst32']
['krne39' 'ffra36' 'dttb37' 'gsga35' 'jyms38']
['iam41' 'wqa42' 'hli44' 'lap43' 'ffs40']


In [36]:
np.linspace(1, 100 - 1, num=11, dtype=int)

array([ 1, 10, 20, 30, 40, 50, 59, 69, 79, 89, 99])

In [43]:
len(list(combinations(ascii_lowercase, 4)))

14950